In [1]:
# clear all the variables
%reset -f
import numpy as np
import pandas as pd
import sys
import turicreate as tc
import os
from os import walk
import scipy.io
import xlrd, os, fnmatch
import array
import pandas as pd
import tensorflow as tf
import chart_studio.plotly as py
import plotly.graph_objs as go
from scipy.spatial.distance import cdist
import plotly.io as pio
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Path variable (where the data is located)
expt_name ='RSVP_spatial_cue_ISI'
myPath = './data/'+expt_name+'/'
if not os.path.exists('./Results/'+expt_name+'/'):
    os.mkdir('./Results/'+expt_name+'/')
resultPath = './Results/'+expt_name+'/'
# Append the path variable to existing search path
sys.path.append(myPath)
# Get the file information in the directory
file_list = []
ignore_list = ['pilot']
for root, dirs, files in os.walk(myPath):
    if 'pilot' in dirs:
        dirs[:]=[]
    for filename in files:
        if fnmatch.fnmatch(filename.lower(), "*csv"):
            file_list.append(os.path.join(root, filename))

/Users/supadhy6/miniconda3/envs/adiLab/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/supadhy6/miniconda3/envs/adiLab/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/supadhy6/miniconda3/envs/adiLab/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/supadhy6/miniconda3/envs/ad

In [2]:
file_list

['./data/RSVP_spatial_cue_extended_letter_duration/10_16Feb2019_10-04.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/11_16Feb2019_10-36.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/12_16Feb2019_11-04.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/13_16Feb2019_11-36.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/14_16Feb2019_14-33.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/15_16Feb2019_15-03.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/16_16Feb2019_15-39.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/17_16Feb2019_16-05.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/18_18Feb2019_14-07.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/19_18Feb2019_14-34.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/1_14Feb2019_14-19.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/20_19Feb2019_09-05.csv',
 './data/RSVP_spatial_cue_extended_letter_duration/21_15Mar2019_15-00.csv',
 './data/RSVP

In [3]:
data = tc.SFrame.read_csv(file_list[0])
print(len(data))
temp = data
for i in range(1,len(file_list)):
    if expt_name == 'RSVP_spatial_cue_replication':
        temp = tc.SFrame.read_csv(file_list[i], column_type_hints = [str,int,int,str,int,int,str,str,int,int,array.array,int,int])
    else:
        temp = tc.SFrame.read_csv(file_list[i], column_type_hints = [str,int,int,str,float,int,str,str,float,float,array.array,int,int])
    #print(len(temp))
    
    print(zip(temp.column_types(),temp.column_names()))
    data = data.append(temp)
len(data)

Finished parsing file /Volumes/My Passport/Github/spatio_temporal_information_processing/RSVP_visual_now/data/RSVP_spatial_cue_extended_letter_duration/10_16Feb2019_10-04.csv

Parsing completed. Parsed 100 lines in 0.035322 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,str,float,int,str,str,float,float,array,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Volumes/My Passport/Github/spatio_temporal_information_processing/RSVP_visual_now/data/RSVP_spatial_cue_extended_letter_duration/10_16Feb2019_10-04.csv

280


...

Finished parsing file /Volumes/My Passport/Github/spatio_temporal_information_processing/RSVP_visual_now/data/RSVP_spatial_cue_extended_letter_duration/8_16Feb2019_09-02.csv

Parsing completed. Parsed 280 lines in 0.009684 secs.

Finished parsing file /Volumes/My Passport/Github/spatio_temporal_information_processing/RSVP_visual_now/data/RSVP_spatial_cue_extended_letter_duration/9_16Feb2019_09-33.csv

Parsing completed. Parsed 280 lines in 0.012617 secs.

6160

In [4]:
data.explore()

Materializing SFrame

In [5]:
import turicreate.aggregate as agg
export = data.groupby(['responsePosRelative0','cueEccentricity1'], operations = {'count':agg.COUNT('responsePosRelative0')})
#data.export_csv(resultPath+'raw_data.csv')
ecc = export['cueEccentricity1'].unique().sort()
sum2 = export[export['cueEccentricity1']==ecc[0]]['count'].sum()
sum10 = export[export['cueEccentricity1']==ecc[1]]['count'].sum()
def get_prob(x, sum2, sum10, ecc):
    if x['cueEccentricity1'] == ecc[0]:
        x['count1'] = x['count']/sum2
    elif x['cueEccentricity1'] == ecc[1]:
        x['count1'] = x['count']/sum10
    return x['count1']
export['count1'] = export.apply(lambda x: get_prob(x,sum2, sum10,ecc))
export = export.sort(['responsePosRelative0'])
#export.export_csv(resultPath+'raw_data.csv')

In [6]:
print(len(data)/280)
import colorlover as cl
colors = cl.scales['3']['seq']['YlOrRd']
colors = colors[0::2]
colors

22.0


['rgb(255,237,160)', 'rgb(240,59,32)']

In [7]:
def add_one(x):
    x['responsePosRelative0'] += 1
    return x['responsePosRelative0']
if expt_name=='RSVP_spatial_cue_extended_letter_duration':
    data['responsePosRelative0'] = data.apply(lambda x: add_one(x) if x['responsePosRelative0']>=0 else x['responsePosRelative0'])
else:
    pass

In [8]:
from plotly import tools

trace = [[]]*2
ecc_length = len(data['cueEccentricity1'].unique())

for i,cuePos in enumerate(data['cueSpatialPos1'].unique()):
    cuePosdata = data[data['cueSpatialPos1']==cuePos]
    temp1 = []
    for j,ecc in enumerate(cuePosdata['cueEccentricity1'].unique().sort()):
        ecc_data = cuePosdata[cuePosdata['cueEccentricity1']==ecc]
        temp1.append(go.Histogram(x = ecc_data['responsePosRelative0'].to_numpy(), 
                                  histnorm='probability',
                                  name = str(ecc) + 'deg', opacity = 1, marker = dict(color = colors[j])))
    trace[i] = temp1

if data['cueSpatialPos1'].unique()[0] == [-1.0,0.0]:
    titles = ('West','East')
else:
    titles = ('East','West')

font_size = 18
fig = tools.make_subplots(rows=1, cols=2, subplot_titles = titles)
fig['layout'].update(barmode='group', title = str(int(len(data)/280))+' participants', font = dict(size = font_size))
#fig['layout']['xaxis3'].update(title='position of reported item relative to the true item', range = [-2,2])
#fig['layout']['xaxis4'].update(title='position of reported item relative to the true item', range = [-2,2])
fig['layout']['xaxis1'].update(title='Serial Position Error (SPE)', titlefont = dict(size = font_size))
fig['layout']['xaxis2'].update(title='Serial Position Error (SPE)', titlefont = dict(size = font_size))
fig['layout']['yaxis1'].update(title='Probability of reports', titlefont = dict(size = font_size))
#fig['layout']['yaxis3'].update(title='Frequency')
for i in range(ecc_length):
    fig.add_trace(trace[0][ecc_length-1-i], 1, 1)
    fig.add_trace(trace[1][ecc_length-1-i], 1, 2)
    #fig.append_trace(trace[2][i], 2, 1)
    #fig.append_trace(trace[3][i], 2, 2)
iplot(fig, filename='SPE_distributions_East_West')

/Users/supadhy6/miniconda3/envs/adiLab/lib/python3.6/site-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



In [9]:
pio.write_image(fig, file=resultPath+'SPE_distributions_East_West.png', format='png')

In [10]:
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde
trace_data = []
X_plot = np.linspace(-5, 5, 1000)
for j,e in enumerate(data['cueEccentricity1'].unique().sort(ascending=False)):
        ecc_data = data[data['cueEccentricity1']==e]
        trace_data.append(go.Histogram(x = ecc_data['responsePosRelative0'].to_numpy(), 
                                  histnorm='probability',name = str(e) + ' deg', opacity = 1, marker = dict(color = colors[1-j], line=dict(color='black',width = 3)), showlegend=False))

fill_colors = ['rgba(255,237,160,0.2)', 'rgba(240,59,32,0.2)']
for j,e in enumerate(data['cueEccentricity1'].unique().sort(ascending=False)):
        ecc_data = data[data['cueEccentricity1']==e]
        X = ecc_data['responsePosRelative0'].to_numpy()
        # Gaussian KDE
        #kde = KernelDensity(kernel='gaussian').fit(X)
        kde = gaussian_kde(X)
        log_dens = kde(X_plot)


        trace_data.append(go.Scatter(x = X_plot, y = log_dens,
                                  mode='markers+lines', fill = 'tozeroy', fillcolor = fill_colors[1-j], name = str(e) + ' deg', 
                                     line = dict(color = colors[1-j], width = 3),
                                     marker=dict(size = 0.5, color='black'),
                                    showlegend=False))

font_size = 34
layout = go.Layout(
    barmode='group',
    font = dict(size = font_size, family = 'Helvetica, Bold'),
    )
    
fig = go.Figure(data=trace_data, layout=layout)
fig['layout'].update(height = 700, width = 850)
fig['layout'].update(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor ='rgba(0,0,0,0)', font = dict(size = font_size, color = 'black'))        

fig.update_xaxes(title = '',showgrid=True, gridwidth=1.5, gridcolor='#DFDFDF', showline=True, linecolor = '#AFAFAF', linewidth = 2.5, range = [-5,5],nticks = 7)
fig.update_yaxes(title = '',showgrid=True, gridwidth=1.5, gridcolor='#DFDFDF', showline=True, linecolor = '#AFAFAF', linewidth = 2.5, nticks = 7)
fig.show()
fig.write_image(resultPath+"SPE_distributions_by_eccentricity.png")
fig.write_image(resultPath+"SPE_distributions_by_eccentricity.svg")
fig.write_image(resultPath+"SPE_distributions_by_eccentricity.pdf")

In [11]:
trace_data = []
distributions = []
fill_colors = ['rgba(255,237,160,0.2)', 'rgba(254,178,76,0.2)', 'rgba(240,59,32,0.2)']
for j,e in enumerate(cuePosdata['cueEccentricity1'].unique().sort()):
        ecc_data = cuePosdata[cuePosdata['cueEccentricity1']==e]
        X = ecc_data['responsePosRelative0'].to_numpy()
        # Gaussian KDE
        #kde = KernelDensity(kernel='gaussian').fit(X)
        kde = gaussian_kde(X)
        log_dens = kde(X_plot)
        distributions.append({'ecc' : e, 'dist' : X, 'dens': log_dens})
        trace_data.append(go.Scatter(x = X_plot, y = log_dens,
                                  mode='lines', fill = 'tozeroy', fillcolor = fill_colors[j], name = str(e) + 'deg', line = dict(color = colors[j], width = 2) ))
        
fig = tools.make_subplots(rows=1, cols=1)
fig['layout'].update(barmode='group', title = str(int(len(data)/280))+' participants', font = dict(size = font_size))
fig['layout']['xaxis1'].update(title='Serial Position Error (SPE) - continuous', titlefont = dict(size = font_size))
fig['layout']['yaxis1'].update(title='Probability of reports', range = [0,0.8], titlefont = dict(size = font_size))
for i in range(ecc_length):
    fig.add_trace(trace_data[i], 1, 1)
iplot(fig, filename='SPE_distributions_curve_fits_zoomed_in')


In [12]:
import numpy as np
from scipy.stats import kurtosis, skew

for x in distributions:
    print( 'Eccentricity: {}'.format( x['ecc']))
    print( 'excess kurtosis of normal distribution (should be 0): {}'.format( kurtosis(x['dens']) ))
    print( 'skewness of normal distribution (should be 0): {}'.format( skew(x['dens']) ))
    print( '\n')

Eccentricity: 2
excess kurtosis of normal distribution (should be 0): -1.4238111047486883
skewness of normal distribution (should be 0): 0.35349657691518854


Eccentricity: 10
excess kurtosis of normal distribution (should be 0): -1.6932455329080731
skewness of normal distribution (should be 0): 0.07449523971011739




In [13]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

hsb = data.to_dataframe()
mod = ols('responsePosRelative0 ~ C(cueEccentricity1)',
                data=hsb).fit()
                
aov_table = sm.stats.anova_lm(mod, typ=2)
print(aov_table)

                           sum_sq      df          F    PR(>F)
C(cueEccentricity1)    133.841558     1.0  10.070595  0.001514
Residual             81841.866883  6158.0        NaN       NaN


In [14]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[1]['ecc']))
scipy.stats.ttest_ind(distributions[0]['dist'], distributions[1]['dist'],axis =0, equal_var=True)

Ecc:2 vs.Ecc:10


Ttest_indResult(statistic=2.889403440500884, pvalue=0.0038867153746432606)

In [15]:
# print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
# scipy.stats.ttest_ind(distributions[0]['dist'], distributions[2]['dist'], axis =0,equal_var=True)

In [16]:
# print('Ecc:{}'.format(distributions[1]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
# scipy.stats.ttest_ind(distributions[1]['dist'], distributions[2]['dist'], axis =0,equal_var=True)

In [17]:
import turicreate.aggregate as agg
sub_data = data.groupby(['subject','cueEccentricity1'],operations = {'avg_lag':agg.AVG('responsePosRelative0'), 'sd_lag':agg.STD('responsePosRelative0')})
ratios = []
approx_ratios = []
c_val = [-1,1]
for sub in data['subject'].unique().sort():
    temp1 = data[data['subject']==sub]
    for ecc in temp1['cueEccentricity1'].unique().sort():
        temp = temp1[temp1['cueEccentricity1']==ecc]
        count_data = temp['responsePosRelative0'].value_counts()
        n = count_data['count'].sum() - count_data[count_data['value']==c_val[0]]['count'].sum()
        a = count_data[count_data['value']==c_val[0]]['count'].to_numpy()
        b = count_data[count_data['value']==c_val[1]]['count'].to_numpy()
        a1 = count_data[count_data['value']<=c_val[0]]['count'].sum()
        b1 = count_data[count_data['value']>=c_val[1]]['count'].sum()
        if len(a) > 0 and len(b)>0:
            ratios.extend(a-b)
        elif len(a)==0 and len(b)>0:
            ratios.extend(0-b)
        elif len(a)>0 and len(b)==0:
            ratios.extend(a-0)
        else:
            ratios.extend(0)
        approx_ratios.extend([(a1-b1)/n])
            
sub_data = sub_data.sort(['subject','cueEccentricity1'])
skewness = []
kurtosissy = []
for sub in data['subject'].unique().sort():
    subd = data[data['subject']==sub]
    for ecc in subd['cueEccentricity1'].unique().sort():
        eccd = subd[subd['cueEccentricity1']==ecc]
        X = eccd['responsePosRelative0'].to_numpy()
        kde = gaussian_kde(X)
        log_dens = kde(X_plot)
        skewness.append(skew(log_dens))
        kurtosissy.append(kurtosis(log_dens))

sub_data['skew'] = skewness
sub_data['kurtosis'] = kurtosissy
sub_data['ratio'] = ratios
sub_data['approx_ratio'] = approx_ratios

In [18]:
sub_hsb = sub_data.to_dataframe()
#sub_hsb = hsb.groupby(['subject','cueEccentricity1']).apply(skew('responseRelative0'))
mod = ols('skew ~ C(cueEccentricity1)',
                data=sub_hsb).fit()
                
aov_table = sm.stats.anova_lm(mod)
aov_table

,df,sum_sq,mean_sq,F,PR(>F)
C(cueEccentricity1),1.0,0.196977,0.196977,1.334676,0.254505
Residual,42.0,6.198536,0.147584,NaN,NaN


In [19]:
mod1 = ols('kurtosis ~ C(cueEccentricity1)',
                data=sub_hsb).fit()
                
aov_table1 = sm.stats.anova_lm(mod1)
aov_table1

,df,sum_sq,mean_sq,F,PR(>F)
C(cueEccentricity1),1.0,0.002496,0.002496,0.006648,0.935404
Residual,42.0,15.771435,0.375510,NaN,NaN


In [20]:
ecc = sub_data['cueEccentricity1'].unique().sort()
deg2_data = sub_data[sub_data['cueEccentricity1']==ecc[0]]
deg2_data = deg2_data.sort('subject')
deg10_data = sub_data[sub_data['cueEccentricity1']==ecc[1]]
deg10_data = deg10_data.sort('subject')

deg2 = deg2_data['avg_lag'].to_numpy()
deg2_sd = deg2_data['sd_lag'].to_numpy()
deg2_ratio = deg2_data['ratio'].to_numpy()
approx_deg2_ratio = deg2_data['approx_ratio'].to_numpy()
deg10 = deg10_data['avg_lag'].to_numpy()
deg10_sd = deg10_data['sd_lag'].to_numpy()
deg10_ratio = deg10_data['ratio'].to_numpy()
approx_deg10_ratio = deg10_data['approx_ratio'].to_numpy()

In [21]:
len(deg2_ratio)

22

In [22]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs. zero mean')
scipy.stats.ttest_1samp(deg2,0.0)

Ecc:2 vs. zero mean


Ttest_1sampResult(statistic=2.7735380152334304, pvalue=0.011384551467244956)

In [23]:
print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[1]['ecc']))
scipy.stats.ttest_rel(deg2, deg10)

Ecc:2 vs.Ecc:10


Ttest_relResult(statistic=3.1573962780268543, pvalue=0.004749986726622105)

In [24]:
# print('Ecc:{}'.format(distributions[0]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
# scipy.stats.ttest_rel(deg2,deg10)

In [25]:
# print('Ecc:{}'.format(distributions[1]['ecc'])+' vs.'+ 'Ecc:{}'.format(distributions[2]['ecc']))
# scipy.stats.ttest_rel(deg6,deg10)

In [26]:
output = tc.SFrame()
output['subject'] = np.arange(1,len(deg2_ratio)+1)
output['avg_lag_2deg'] = np.round(deg2,2)
output['sd_lag_2deg'] = np.round(deg2_sd,2)
output['avg_lag_10deg'] = np.round(deg10,2)
output['sd_lag_10deg'] = np.round(deg10_sd,2)

#output['ratio_2deg'] = deg2_ratio
output['N_l_minus_r_2deg'] = approx_deg2_ratio
#output['ratio_10deg'] = deg10_ratio
output['N_l_minus_r_10deg'] = approx_deg10_ratio
output['past'] = output.apply(lambda x: 1 if x['avg_lag_2deg']>x['avg_lag_10deg'] else 0)
output

subject,avg_lag_2deg,sd_lag_2deg,avg_lag_10deg,sd_lag_10deg,N_l_minus_r_2deg,N_l_minus_r_10deg,past
1,0.39,2.1,0.06,3.1,-0.23404255319148937,0.17073170731707318,1
2,0.6,7.03,-0.41,7.0,0.018691588785046728,0.3302752293577982,1
3,0.69,3.41,0.15,3.52,-0.3883495145631068,-0.24761904761904763,1
4,0.27,1.46,-0.08,1.91,-0.3146067415730337,-0.1348314606741573,1
5,-0.86,2.77,-0.73,2.34,0.8253968253968254,1.6923076923076923,0
6,0.54,3.87,-0.11,4.0,-0.5254237288135594,-0.27522935779816515,1
7,0.66,2.7,0.62,3.26,-0.5391304347826087,-0.40384615384615385,1
8,0.51,2.73,0.57,3.11,-0.5636363636363636,-0.5272727272727272,0
9,0.32,1.96,0.56,3.07,-0.36538461538461536,-0.28846153846153844,0
10,0.27,2.42,-0.49,2.91,-0.09302325581395349,0.647887323943662,1


In [27]:
import pandas as pd
def highlight_cells(x):
    # provide your criteria for highlighting the cells here
    threshold = 3.5*x.std()
    outlier = x>=threshold
    return ['background-color: red' if v else '' for v in outlier]

#output.export_csv(filename = resultPath+'subject_level_stats.csv')
output_df = output.to_dataframe()

output_df.style.\
    apply(highlight_cells).\
    to_excel(resultPath+expt_name+'subject_level_stats.xlsx', engine='openpyxl')

In [28]:
# print('Ecc:{}'.format(2)+' vs.'+ 'Ecc:{}'.format(6))
# scipy.stats.ttest_rel(output['avg_lag_2deg'].to_numpy(), output['avg_lag_6deg'].to_numpy())

In [29]:
print('Ecc:{}'.format(ecc[0])+' vs.'+ 'Ecc:{}'.format(ecc[1]))
avg_t, avg_p = scipy.stats.ttest_rel(output['avg_lag_2deg'].to_numpy(), output['avg_lag_10deg'].to_numpy())
round(avg_p,3)

Ecc:2 vs.Ecc:10


0.005

In [30]:
# print('Ecc:{}'.format(6)+' vs.'+ 'Ecc:{}'.format(10))
# scipy.stats.ttest_rel(output['avg_lag_6deg'].to_numpy(), output['avg_lag_10deg'].to_numpy())

In [31]:
# print('Ecc:{}'.format(2)+' vs.'+ 'Ecc:{}'.format(6))
# scipy.stats.ttest_rel(output['ratio_2deg'].to_numpy(), output['ratio_6deg'].to_numpy())

In [32]:
# print('Ecc:{}'.format(ecc[0])+' vs.'+ 'Ecc:{}'.format(ecc[1]))
# ratio_t, ratio_p = scipy.stats.ttest_rel(output['ratio_2deg'].to_numpy(), output['ratio_10deg'].to_numpy())
# ratio_p

In [33]:
print('Ecc:{}'.format(ecc[0])+' vs.'+ 'Ecc:{}'.format(ecc[1]))
approx_ratio_t, approx_ratio_p = scipy.stats.ttest_rel(output['N_l_minus_r_2deg'].to_numpy(), output['N_l_minus_r_10deg'].to_numpy())
approx_ratio_p

Ecc:2 vs.Ecc:10


9.999318633545065e-05

In [34]:
# trace_data = []
# trace_data.append(go.Bar(x = [str(ecc[0])+' deg'],
#                         y = [output['ratio_2deg'].mean()],
#                         name=str(ecc[0])+' deg',
#                         marker = dict(color = colors[0]),
#                         error_y = dict(type = 'data',
#                                       array = [output['ratio_2deg'].std()/np.sqrt(len(output))],
#                                       visible=True
#                                       )
#                         )
#                  )

# # trace_data.append(go.Bar(x = ['preference to lag -1 over lag 1 responses'],
# #                         y = [output['ratio_6deg'].mean()],
# #                         name='6 deg',
# #                         marker = dict(color = colors[1]),
# #                         error_y = dict(type = 'data',
# #                                       array = [output['ratio_6deg'].std()/np.sqrt(14)],
# #                                       visible=True
# #                                       )
# #                         )
# #                  )

# trace_data.append(go.Bar(x = [str(ecc[1]) +' deg'],
#                         y = [output['ratio_10deg'].mean()],
#                         name=str(ecc[1])+' deg',
#                         marker = dict(color = colors[1]),
#                         error_y = dict(type = 'data',
#                                       array = [output['ratio_10deg'].std()/np.sqrt(len(output))],
#                                       visible=True
#                                       )
#                         )
#                  )

# y_max = max(output['ratio_10deg'].mean(), output['ratio_2deg'].mean(), 
#                    output['ratio_10deg'].mean()+output['ratio_10deg'].std()/np.sqrt(len(output)),
#                   output['ratio_2deg'].mean()+output['ratio_2deg'].std()/np.sqrt(len(output))) + 5

# trace_data.append(go.Scatter(x = [str(i)+' deg' for i in ecc],
#                             y = [y_max for i in range(len(ecc))],
#                             mode='markers+lines',
#                             marker = dict(color='black',size = 15, symbol='line-ns-open'),
#                             line = dict(color='black',width = 2),
#                             showlegend = False))

# for j,e in enumerate(cuePosdata['cueEccentricity1'].unique().sort(ascending=False)):
#         ecc_data = cuePosdata[cuePosdata['cueEccentricity1']==e]
#         trace_data.append(go.Histogram(x = ecc_data['responsePosRelative0'].to_numpy(), 
#                                   histnorm='probability',name = str(e) + 'deg', opacity = 1, marker = dict(color = colors[1-j]),
#                                       xaxis = 'x2',
#                                       yaxis = 'y2',
#                                       showlegend=False))

# pval_text = ['** p = '+str(np.round(ratio_p,3))+ '  (< 0.05)' if ratio_p < 0.05 else 'n.s']
# layout = go.Layout(
#     barmode='group',
#     width = 800,
#     height = 600,
#     title = str(int(len(data)/280))+' participants',
#     xaxis = dict(zeroline=False, title = 'Probe spatial separation from the RSVP stream', range = [-1,4]),
#     yaxis = dict(title = 'SPE('+str(c_val[0])+') - SPE('+str(c_val[1])+') responses',zeroline=False, range = [-30,45]),
#     annotations=[dict(x=0.5,y=y_max+10,xref='x',yref='y',text=pval_text[0], showarrow=False, font=dict(family = 'Courier New, monospace',
#                                                                                                      size = 20))],
#     xaxis2 = dict(domain = [0.7, 1], anchor = 'y2', range = [-5,5], title = 'SPE', showline=True, mirror=True),
#     yaxis2 = dict(domain = [0.55, 0.9], anchor = 'x2', title = 'Prob.', showline=True, mirror=True)
#     )
    
# fig = go.Figure(data=trace_data, layout=layout)
# iplot(fig, filename='SPE('+str(c_val[0])+') - SPE('+str(c_val[1])+')'+'_comparision')

In [35]:
#pio.write_image(fig, file=resultPath+'SPE('+str(c_val[0])+') - SPE('+str(c_val[1])+')'+'_comparision.png', format='png')

In [36]:
trace_data = []
trace_data.append(go.Bar(x = [str(ecc[0])+' deg'],
                        y = [output['N_l_minus_r_2deg'].mean()],
                        name=str(ecc[0])+' deg',
                        marker = dict(color = colors[0]),
                        error_y = dict(type = 'data',
                                      array = [output['N_l_minus_r_2deg'].std()/np.sqrt(len(output))],
                                      visible=True
                                      )
                        )
                 )

# trace_data.append(go.Bar(x = ['preference to lag -1 over lag 1 responses'],
#                         y = [output['ratio_6deg'].mean()],
#                         name='6 deg',
#                         marker = dict(color = colors[1]),
#                         error_y = dict(type = 'data',
#                                       array = [output['ratio_6deg'].std()/np.sqrt(14)],
#                                       visible=True
#                                       )
#                         )
#                  )

trace_data.append(go.Bar(x = [str(ecc[1]) +' deg'],
                        y = [output['N_l_minus_r_10deg'].mean()],
                        name=str(ecc[1])+' deg',
                        marker = dict(color = colors[1]),
                        error_y = dict(type = 'data',
                                      array = [output['N_l_minus_r_10deg'].std()/np.sqrt(len(output))],
                                      visible=True
                                      )
                        )
                 )

y_max = max(output['N_l_minus_r_10deg'].mean(), output['N_l_minus_r_2deg'].mean(), 
                   output['N_l_minus_r_10deg'].mean()+output['N_l_minus_r_10deg'].std()/np.sqrt(len(output)),
                  output['N_l_minus_r_2deg'].mean()+output['N_l_minus_r_2deg'].std()/np.sqrt(len(output))) + 0.15

trace_data.append(go.Scatter(x = [str(i)+' deg' for i in ecc],
                            y = [y_max for i in range(len(ecc))],
                            mode='markers+lines',
                            marker = dict(color='black',size = 15, symbol='line-ns-open'),
                            line = dict(color='black',width = 2),
                            showlegend = False))

# for j,e in enumerate(cuePosdata['cueEccentricity1'].unique().sort(ascending=False)):
#         ecc_data = cuePosdata[cuePosdata['cueEccentricity1']==e]
#         trace_data.append(go.Histogram(x = ecc_data['responsePosRelative0'].to_numpy(), 
#                                   histnorm='probability',name = str(e) + 'deg', opacity = 1, marker = dict(color = colors[1-j]),
#                                       xaxis = 'x2',
#                                       yaxis = 'y2',
#                                       showlegend=False))

pval_text = ['** p = '+str(np.round(approx_ratio_p,3))+ '  (< 0.05)' if approx_ratio_p < 0.05 else 'n.s']

annotations = [dict(x=0.5,y=y_max+0.2,xref='x',yref='y',text=pval_text[0], showarrow=False, font=dict(family = 'Courier New, monospace',
                                                                                                     size = 20))]

ystart = 0
ymax1 = 0
ymin1 = 1
padding = 0.25
x_padding = 0.025
xpos1 = 0.1

annotations.append(dict(y=ymax1, x=xpos1+x_padding, ay=ymin1 - padding, ax=xpos1+x_padding, xref='x', axref='x', xshift = -155,
            yref='y', ayref='y', showarrow=True, arrowhead=1, arrowside='start',arrowsize=1, arrowwidth=1, arrowcolor='gray', text = 'past', font = dict(size = 15, color='gray')))

layout = go.Layout(
    barmode='group',
    width = 1200,
    height = 800,
    font = dict(size = font_size),
    title = str(int(len(data)/280))+' participants',
    xaxis = dict(zeroline=False, title = '', range = [-1.5,2.5], titlefont = dict(size = font_size)),
    yaxis = dict(title = '',zeroline=False, range = [-1,1], titlefont = dict(size = font_size)),
    annotations= annotations,
    xaxis2 = dict(domain = [0.7, 1], anchor = 'y2', title = 'SPE', showline=True, mirror=True, zeroline=False, titlefont = dict(size = 14)),
    yaxis2 = dict(domain = [0.55, 0.9], anchor = 'x2', title = 'Prob.', showline=True, mirror=True, zeroline=False, titlefont = dict(size = 14))
    )
    
fig = go.Figure(data=trace_data, layout=layout)
iplot(fig, filename='avg#_of_(left - right)_SPEs_from_0_comparision')

In [37]:
pio.write_image(fig, file=resultPath+'avg#_of_(left - right)_SPEs_from_0_comparision.png', format='png')

In [38]:
trace_data = []
trace_data.append(go.Bar(y = [str(ecc[0])+' deg'],
                        x = [output['avg_lag_2deg'].mean()], orientation='h',
                        name=str(ecc[0])+' deg',
                        marker = dict(color = colors[0], line = dict(color='black',width=3)),
                        error_x = dict(type = 'data',
                                      array = [output['avg_lag_2deg'].std()/np.sqrt(len(output))],
                                      visible=True
                                      ), showlegend=False
                        )
                 )


trace_data.append(go.Bar(y = [str(ecc[1]) +' deg'],
                        x = [output['avg_lag_10deg'].mean()], orientation ='h',
                        name=str(ecc[1])+' deg',
                        marker = dict(color = colors[1], line = dict(color = 'black', width = 3)),
                        error_x = dict(type = 'data',
                                      array = [output['avg_lag_10deg'].std()/np.sqrt(len(output))],
                                      visible=True
                                      ),showlegend=False
                        )
                 )

x_max = -max(output['avg_lag_10deg'].mean(), output['avg_lag_2deg'].mean(), 
                   output['avg_lag_10deg'].mean()+output['avg_lag_10deg'].std()/np.sqrt(len(output)),
                  output['avg_lag_2deg'].mean()+ output['avg_lag_2deg'].std()/np.sqrt(len(output))) + 0.2

trace_data.append(go.Scatter(y = [str(i)+' deg' for i in ecc],
                            x = [x_max for i in range(len(ecc))],
                            mode='markers+lines',
                            marker = dict(color='black',size = 25, symbol='line-ew-open'),
                            line = dict(color='black',width = 2.5),
                            showlegend = False))


pval_text = ['** p = '+str(np.round(avg_p,3)) if round(avg_p,3) < 0.05 else 'n.s']

xstart = 0
xmax1 = 0.35
xmin1 = -0.35
padding = 0.2
ypos1 = 0.0

annotations = [dict(y=0.5,x=x_max-0.1,xref='x',yref='y',text=pval_text[0], showarrow=False, font=dict(family = 'Helvetica, bold',
                                                                                                     size = 20))]
# annotations.append(dict(x=xmax1-padding, y=ypos1-padding, ax=xmin1+padding, ay=ypos1-padding, xref='x', axref='x', yshift = -100,
#             yref='y', ayref='y', showarrow=True, arrowhead=1, arrowside='start',arrowsize=1.5, arrowwidth=3.5, arrowcolor='black', text = 'past', font = dict(size = 20, color='black')))
layout = go.Layout(
    barmode='group',
    width = 1200,
    height = 800,
    font = dict(size = font_size-3),
    #title = str(int(len(data)/280))+' participants',
    yaxis = dict(title = '', range = [-1,2], titlefont = dict(size = font_size)),
    xaxis = dict(title = '', range = [-0.5,0.5],titlefont = dict(size = font_size)),
    annotations= annotations,
    #xaxis2 = dict(domain = [0.6, 0.95], anchor = 'y2', range = [-5,5], title = 'SPE', showline=True, mirror=True, titlefont = dict(size = 14)),
    #yaxis2 = dict(domain = [0.6, 0.95], anchor = 'x2', title = 'Prob.', showline=True, mirror=True, titlefont = dict(size = 14))
    )
    
fig = go.Figure(data=trace_data, layout=layout)
fig['layout'].update(height = 800, width = 850)
fig['layout'].update(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor ='rgba(0,0,0,0)', font = dict(size = 34, color = 'black'))        



fig.update_xaxes(showgrid=True, gridwidth=1.5, gridcolor='#DFDFDF', showline=True, linecolor = '#AFAFAF', zeroline=True, zerolinecolor = 'black',zerolinewidth = 2.5, linewidth = 2.5, nticks = 7)
fig.update_yaxes(showgrid=True, gridwidth=1.5, gridcolor='#DFDFDF', showline=True, linecolor = '#AFAFAF', linewidth = 2.5)
fig.show()
fig.write_image(resultPath+"avg_SPE_comparison.png")
fig.write_image(resultPath+"avg_SPE_comparison.svg")
fig.write_image(resultPath+"avg_SPE_comparison.pdf")

In [39]:
pio.write_image(fig, file=resultPath+'avg_SPE_comparision.png', format='png')